In [2]:
# Datamart server running on localhost in development mode
datamart_api_url = 'http://localhost:5000'

In [3]:
from requests import get,post,put,delete
import json
import pandas as pd
from io import StringIO
from IPython.display import display, HTML

In [4]:
import os
def upload_data_post(file_path, url):
    file_name = os.path.basename(file_path)
    files = {
        'file': (file_name, open(file_path, mode='rb'), 'application/octet-stream')
    }
    response = post(url, files=files)

    print(json.dumps(response.json(), indent=2))

In [5]:
response = get(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables')
print(json.dumps(response.json(), indent=2))


[]


In [8]:
url = f'{datamart_api_url}/datasets/DeleteMe/annotated'
file_path = 'test/test_data/03_worker_incidents_tbd.xlsx'
upload_data_post(file_path, url)

[
  {
    "Error": "No variable un in dataset DeleteMe"
  },
  {
    "Error": "No variable ingo in dataset DeleteMe"
  }
]


In [39]:
response = get(f'{datamart_api_url}/datasets/DeleteMe/variables/un')
df = pd.read_csv(StringIO(response.text))
display(HTML(df.fillna('').to_html(index=False)))

location_id,dataset_id,variable_id,variable,main_subject,main_subject_id,value,value_unit,time,time_precision,country,admin1,admin2,admin3,coordinate,stated_in,stated_in_id,location,stated in


In [37]:
response = delete(f'{datamart_api_url}/datasets/DeleteMe/variables')
print(response.text)
# print(json.dumps(response.json(), indent=2))

{"message": "The method is not allowed for the requested URL."}



In [48]:
response = delete(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables/ingo')
print(response.text)
# print(json.dumps(response.json(), indent=2))

{"Message": "Variable: ingo in the dataset: DeleteMe has been deleted."}



In [49]:
response = get(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables')
print(len(response.json()))
print(json.dumps(response.json(), indent=2)) # print only 5 variables

0
[]
